In [ ]:
# To read the PDF
import PyPDF2
# To analyze the PDF layout and extract text
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure
# To extract text from tables in PDF
import pdfplumber
#file management
import os
import sys
import re

The code adapts to the post: https://towardsdatascience.com/extracting-text-from-pdf-files-with-python-a-comprehensive-guide-9fc4003d517

In [ ]:
print('current directory: ' + os.getcwd())
data_path='../data/demo_data/'
files = [f for f in os.listdir(data_path) if f.endswith('.pdf')]
print('demofiles:')
for file in files:
    print(file)

Text extraction

In [ ]:
# Create a function to extract text

def text_extraction(element):
    # Extracting the text from the in-line text element
    line_text = element.get_text()
    
    # Find the formats of the text
    # Initialize the list with all the formats that appeared in the line of text
    line_formats = []
    for text_line in element:
        if isinstance(text_line, LTTextContainer):
            # Iterating through each character in the line of text
            for character in text_line:
                if isinstance(character, LTChar):
                    # Append the font name of the character and the size
                    line_formats.append((character.fontname,character.size))
                    # Append the font size of the character
                    #line_formats.append(character.size)
    # Find the unique font sizes and names in the line
    if len(line_formats)>0:
        format_per_line = max(set(line_formats), key=line_formats.count) #list(set(line_formats))
    else:
        format_per_line=[]
    
    # Return a tuple with the text in each line along with its format
    return (line_text, format_per_line)

In [ ]:
def remove_header_footer(pdf_extracted_text):
        page_format_pattern = r'([page]+[\d]+)'
        #pdf_extracted_text = pdf_extracted_text.lower().split("\n")
        header = pdf_extracted_text[0].strip().lower()
        footer = pdf_extracted_text[-1].strip().lower()
        if re.search(page_format_pattern, header) or header.isnumeric():
            pdf_extracted_text = pdf_extracted_text[1:]
        if re.search(page_format_pattern, footer) or footer.isnumeric():
            pdf_extracted_text = pdf_extracted_text[:-1]
        pdf_extracted_text = "\n".join(pdf_extracted_text)
        return pdf_extracted_text

In [ ]:
pdf_path = data_path+files[1]
# create a PDF file object
pdfFileObj = open(pdf_path, 'rb')
# create a PDF reader object
pdfReaded = PyPDF2.PdfReader(pdfFileObj)

# Create the dictionary to extract text from each image
text_per_page = {}

In [ ]:
reader = PyPDF2.PdfReader(pdf_path)
reader.metadata
len(reader.pages)


In [ ]:
all_formats = []
for pagenum, page in enumerate(extract_pages(pdf_path)):
    # Initialize the variables needed for the text extraction from the page
    pageObj = pdfReaded.pages[pagenum]
    page_text = []
    line_format = []
    text_from_images = []
    text_from_tables = []
    page_content = []
    # Initialize the number of the examined tables
    table_num = 0
    first_element= True
    table_extraction_flag= False
    # Open the pdf file
    pdf = pdfplumber.open(pdf_path)
    # Find the examined page
    page_tables = pdf.pages[pagenum]
    # Find the number of tables on the page
    tables = page_tables.find_tables()


    # Find all the elements
    page_elements = [(element.y1, element) for element in page._objs]
    # Sort all the elements as they appear in the page 
    page_elements.sort(key=lambda a: a[0], reverse=True)

    # Find the elements that composed a page
    for i,component in enumerate(page_elements):
        # Extract the position of the top side of the element in the PDF
        pos= component[0]
        # Extract the element of the page layout
        element = component[1]
        
        # Check if the element is a text element
        if isinstance(element, LTTextContainer):
            # Check if the text appeared in a table
            if table_extraction_flag == False:
                # Use the function to extract the text and format for each text element
                (line_text, format_per_line) = text_extraction(element)
                # Append the text of each line to the page text
                page_text.append(line_text)
                # Append the format for each line containing text
                line_format.append(format_per_line)
                page_content.append(line_text)
            else:
                # Omit the text that appeared in a table
                pass
    #exclude headers and page numbers
    page_content = remove_header_footer(page_content)
    # Create the key of the dictionary
    dctkey = 'Page_'+str(pagenum)
    all_formats.append(line_format)
    # Add the list of list as the value of the page key
    text_per_page[dctkey]= [page_text, line_format, text_from_images,text_from_tables, page_content]

# Closing the pdf file object
pdfFileObj.close()

# Deleting the additional files created
#os.remove('cropped_image.pdf')
#os.remove('PDF_image.png')

# Display the content of the page
result = ''.join(text_per_page['Page_0'][4])
print(result)


Find the most common font

In [ ]:
flat_list = [item for sublist in all_formats for item in sublist]
most_common_format = max(flat_list, key=flat_list.count)
print(most_common_format)